# Sklearn モデルのデプロイ
推論Dockerイメージを作成し、Azure Container Instancesにデプロイします。

## Python SDK バージョン確認

In [2]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.10


## Workspace情報の取得

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

exp = Experiment(workspace = ws, name = "devops-notebook")

Found the config file in: /Users/konabuta/Project/DevOpsLab/code/aml_config/config.json
Workspace name: azureml
Azure region: southeastasia
Subscription id: 9c0f91b8-eb2f-484c-979c-15848c098a6b
Resource group: azureml_sea


## モデル取得

In [56]:
from azureml.core.model import Model
model=Model(ws, 'diabetes-notebook')

In [21]:
import os 
script_folder = "scoring"
os.makedirs(script_folder, exist_ok=True)

In [22]:
%%writefile {script_folder}/score_diabetes.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('diabetes-notebook')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Writing scoring/score_diabetes.py


In [25]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open(script_folder+"/myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [26]:
with open(script_folder+"/myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- scikit-learn



In [27]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "diabetes",  "method" : "sklearn"}, 
                                               description='Predict diabetes with sklearn')

## 推論Dockerイメージ

### Dockerイメージ設定

In [53]:
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

os.chdir("scoring")

# 推論Dockerイメージの設定
image_config = ContainerImage.image_configuration(execution_script="score_diabetes.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

os.chdir("..")

### Dockerイメージ作成

In [ ]:
%%time
from azureml.core.image import Image,ContainerImage

image = Image.create(name = "sklearn-diabete-image",
                     models = [model],
                     image_config = image_config,
                     workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running.........................

In [ ]:
print('{}(v.{} [{}]) stored at {} with build log {}'.format(image.name, image.version, image.creation_state, image.image_location, image.image_build_log_uri))

In [ ]:
%%time
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                            memory_gb=1, 
                                            tags={'area': "diabetes", 'type': "regression"},
                                            description='A diabetes model from jupyter')

service = Webservice.deploy_from_image(deployment_config=aciconfig,
                                        image=image,
                                        name="sklearn-diabetes-svc",
                                        workspace=ws)

service.wait_for_deployment(show_output=True)